In [1]:
#subodh Kumar 
#Predicting t20 match prediction 
import numpy as np
import pandas as pd
from yaml import safe_load
import os
from tqdm import tqdm

In [2]:
# filenames = []
# for file in os.listdir("C:\Users\hp\Downloads\world-t20-score-predictor"):
#     filenames.append(os.path.join("C:\Users\hp\Downloads\world-t20-score-predictor",file))
    
    
import os
folder_path = r"C:\Users\hp\Downloads\world-t20-score-predictor"  # Use raw string
filenames = [os.path.join(folder_path, file) for file in os.listdir(folder_path)]

#print(filenames)  # Check output


In [3]:
filenames[0:5]


['C:\\Users\\hp\\Downloads\\world-t20-score-predictor\\1001349.yaml',
 'C:\\Users\\hp\\Downloads\\world-t20-score-predictor\\1001351.yaml',
 'C:\\Users\\hp\\Downloads\\world-t20-score-predictor\\1001353.yaml',
 'C:\\Users\\hp\\Downloads\\world-t20-score-predictor\\1004729.yaml',
 'C:\\Users\\hp\\Downloads\\world-t20-score-predictor\\1007655.yaml']

In [4]:
final_df = pd.DataFrame()
counter = 1
for file in tqdm(filenames):
    with open(file, 'r') as f:
        df = pd.json_normalize(safe_load(f))
        df['match_id'] = counter
        final_df = final_df.append(df)
        counter+=1
        
final_df

  0%|                                                                                         | 0/3858 [00:00<?, ?it/s]C:\Users\hp\AppData\Local\Temp\ipykernel_10928\2042649833.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(df)
  0%|                                                                                 | 1/3858 [00:00<44:52,  1.43it/s]C:\Users\hp\AppData\Local\Temp\ipykernel_10928\2042649833.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(df)
  0%|                                                                                 | 2/3858 [00:01<42:27,  1.51it/s]C:\Users\hp\AppData\Local\Temp\ipykernel_10928\2042649833.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import json  # Use json instead of yaml.safe_load for JSON files
from tqdm import tqdm

final_dfs = []  # Use list for efficient concatenation
counter = 1

for file in tqdm(filenames):
    with open(file, 'r', encoding='utf-8') as f:
        df = pd.json_normalize(safe_load(f))  # Normalize nested JSON
        df['match_id'] = counter
        final_dfs.append(df)  # Store in a list
        counter += 1

final_df = pd.concat(final_dfs, ignore_index=True)  # Concatenate once

print(final_df.shape)  # Check final dataframe size


In [ ]:
backup = final_df.copy()


In [ ]:
final_df


In [ ]:
final_df.drop(columns=[
    'meta.data_version',
    'meta.created',
    'meta.revision',
    'info.outcome.eliminator',
    'info.outcome.result',
    'info.outcome.method',
    'info.neutral_venue',
    'info.match_type_number',
    'info.outcome.by.runs',
    'info.outcome.by.wickets'
],inplace=True)
#info.outcome.bowl_out', 'info.bowl_out', 'info.supersubs.South Africa', 'info.supersubs.New Zealand'] not found in axis"

In [ ]:
#subodh kumar
final_df


In [ ]:
final_df['info.gender'].value_counts()


In [ ]:
final_df = final_df[final_df['info.gender'] == 'male']
final_df.drop(columns=['info.gender'],inplace=True)
final_df

In [ ]:
final_df['info.match_type'].value_counts()


In [ ]:
final_df['info.overs'].value_counts()


In [ ]:
import pickle
pickle.dump(final_df,open('dataset_level1.pkl','wb'))

In [ ]:
matches = pickle.load(open('dataset_level1.pkl','rb'))
matches.iloc[0]['innings'][0]['1st innings']['deliveries']

In [ ]:
count = 1
delivery_df = pd.DataFrame()
for index, row in matches.iterrows():
    if count in [75,108,150,180,268,360,443,458,584,748,982,1052,1111,1226,1345]:
        count+=1
        continue
    count+=1
    ball_of_match = []
    batsman = []
    bowler = []
    runs = []
    player_of_dismissed = []
    teams = []
    batting_team = []
    match_id = []
    city = []
    venue = []
    for ball in row['innings'][0]['1st innings']['deliveries']:
        for key in ball.keys():
            match_id.append(count)
            batting_team.append(row['innings'][0]['1st innings']['team'])
            teams.append(row['info.teams'])
            ball_of_match.append(key)
            batsman.append(ball[key]['batsman'])
            bowler.append(ball[key]['bowler'])
            runs.append(ball[key]['runs']['total'])
            city.append(row['info.city'])
            venue.append(row['info.venue'])
            try:
                player_of_dismissed.append(ball[key]['wicket']['player_out'])
            except:
                player_of_dismissed.append('0')
    loop_df = pd.DataFrame({
            'match_id':match_id,
            'teams':teams,
            'batting_team':batting_team,
            'ball':ball_of_match,
            'batsman':batsman,
            'bowler':bowler,
            'runs':runs,
            'player_dismissed':player_of_dismissed,
            'city':city,
            'venue':venue
        })
    delivery_df = delivery_df.append(loop_df)

In [ ]:
delivery_df

In [ ]:
def bowl(row):
    for team in row['teams']:
        if team!=row['batting_team']:
            return team

In [ ]:
delivery_df['bowling_team']=delivery_df.apply(bowl,axis=1)

In [ ]:
delivery_df

In [ ]:
delivery_df.drop(columns=['teams'],inplace=True)

In [39]:
delivery_df

,match_id,batting_team,ball,batsman,bowler,runs,player_dismissed,city,venue,bowling_team
0,2,Australia,0.1,AJ Finch,SL Malinga,0,0,NaN,Melbourne Cricket Ground,Sri Lanka
1,2,Australia,0.2,AJ Finch,SL Malinga,0,0,NaN,Melbourne Cricket Ground,Sri Lanka
2,2,Australia,0.3,AJ Finch,SL Malinga,1,0,NaN,Melbourne Cricket Ground,Sri Lanka
3,2,Australia,0.4,M Klinger,SL Malinga,2,0,NaN,Melbourne Cricket Ground,Sri Lanka
4,2,Australia,0.5,M Klinger,SL Malinga,0,0,NaN,Melbourne Cricket Ground,Sri Lanka
...,...,...,...,...,...,...,...,...,...,...
116,188,Jersey,19.2,EJB Miles,J Jojo,1,0,Castel,King George V Sports Ground,Denmark
117,188,Jersey,19.3,CW Perchard,J Jojo,2,EJB Miles,Castel,King George V Sports Ground,Denmark
118,188,Jersey,19.4,WJR Robertson,J Jojo,0,0,Castel,King George V Sports Ground,Denmark
119,188,Jersey,19.5,WJR Robertson,J Jojo,1,0,Castel,King George V Sports Ground,Denmark


In [40]:
delivery_df['batting_team'].unique()

array(['Australia', 'Hong Kong', 'Zimbabwe', 'India', 'Bangladesh',
       'New Zealand', 'South Africa', 'England', 'West Indies',
       'Pakistan', 'Scotland', 'Oman', 'Ireland', 'Papua New Guinea',
       'United Arab Emirates', 'Sri Lanka', 'Netherlands', 'Nepal',
       'Vanuatu', 'Philippines', 'United States of America', 'Germany',
       'Ghana', 'Uganda', 'Kenya', 'Namibia', 'Nigeria', 'Botswana',
       'Guernsey', 'Denmark', 'Jersey'], dtype=object)

In [41]:
teams=[
    'Austrailia',
    'Zimbabwe',
    'India',
    'Bangladesh',
    'New Zealand',
    'South Africa',
    'England', 
    'West Indies',
    'Pakistan', 
    'United Arab Emirates', 
    'Sri Lanka', 
    'Netherlands'
]

In [42]:
delivery_df=delivery_df[delivery_df['batting_team'].isin(teams)]
delivery_df=delivery_df[delivery_df['bowling_team'].isin(teams)]


In [43]:
delivery_df

,match_id,batting_team,ball,batsman,bowler,runs,player_dismissed,city,venue,bowling_team
0,6,Zimbabwe,0.1,CJ Chibhabha,JD Unadkat,0,0,NaN,Harare Sports Club,India
1,6,Zimbabwe,0.2,CJ Chibhabha,JD Unadkat,0,0,NaN,Harare Sports Club,India
2,6,Zimbabwe,0.3,CJ Chibhabha,JD Unadkat,0,0,NaN,Harare Sports Club,India
3,6,Zimbabwe,0.4,CJ Chibhabha,JD Unadkat,1,0,NaN,Harare Sports Club,India
4,6,Zimbabwe,0.5,H Masakadza,JD Unadkat,0,0,NaN,Harare Sports Club,India
...,...,...,...,...,...,...,...,...,...,...
74,147,West Indies,12.2,OC McCoy,AU Rashid,0,0,Basseterre,"Warner Park, St Kitts",England
75,147,West Indies,12.3,OC McCoy,AU Rashid,6,0,Basseterre,"Warner Park, St Kitts",England
76,147,West Indies,12.4,OC McCoy,AU Rashid,4,0,Basseterre,"Warner Park, St Kitts",England
77,147,West Indies,12.5,OC McCoy,AU Rashid,0,0,Basseterre,"Warner Park, St Kitts",England


In [44]:
output= delivery_df[['match_id','batting_team','bowling_team','ball','runs','player_dismissed','city','venue']]

In [45]:
output

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,6,Zimbabwe,India,0.1,0,0,NaN,Harare Sports Club
1,6,Zimbabwe,India,0.2,0,0,NaN,Harare Sports Club
2,6,Zimbabwe,India,0.3,0,0,NaN,Harare Sports Club
3,6,Zimbabwe,India,0.4,1,0,NaN,Harare Sports Club
4,6,Zimbabwe,India,0.5,0,0,NaN,Harare Sports Club
...,...,...,...,...,...,...,...,...
74,147,West Indies,England,12.2,0,0,Basseterre,"Warner Park, St Kitts"
75,147,West Indies,England,12.3,6,0,Basseterre,"Warner Park, St Kitts"
76,147,West Indies,England,12.4,4,0,Basseterre,"Warner Park, St Kitts"
77,147,West Indies,England,12.5,0,0,Basseterre,"Warner Park, St Kitts"


In [48]:
pickle.dump(output,open('dataset_level2.pkl','wb'))